# Loading Saved NN Algorithm in Order to Make New Predictions

By: Matt Purvis

This project will perform data preparation on a new dataset and load a previously trained neural network to perform predictions on a new dataset.  

In [2]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
from fuzzywuzzy import fuzz
import math
from collections import OrderedDict
import pickle
import time
from datetime import date, timedelta
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from tensorflow.keras.models import save_model, load_model

# Read in the Data

In [4]:
# Set Date Variables to load the correct files
today = date.today() - timedelta(days = 2)
todayds = today.strftime("%Y-%m-%d")
todayds2 = today.strftime("%Y_%m_%d")
yesterdate = today - timedelta(days = 1)

In [11]:
#Import Files
todaydvp = pd.read_csv("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\TodayDVP\\draftkings_NBA_" + todayds + "_players.csv")
common_lineups = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\DepthCharts\\Basketball_Monster_Rotation.csv')
common_lineups2 = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\DepthCharts\\BBM_Common_lineups.csv')
depth_charts = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\DepthCharts\\BBM_depth_charts.csv')
injuriestwitter =  pd.read_csv("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Injuries\\injuriestwitter.csv")
injuries = pd.read_csv("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Injuries\\injuries.csv")
bangedup =  pd.read_csv("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Injuries\\bangedup.csv")
bangeduptwitter = pd.read_csv("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Injuries\\bangeduptwitter.csv")
firstgame = pd.read_csv("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Original Files\\draftkings_NBA_2021-01-22_players.csv")
Pace = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Pace\\pace.csv')
SS = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\SS_Projections.csv')
usage = pd.read_excel('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Usage\\BBM_PlayerRankings.xls')
usage = usage.sort_values(by = ['Team', 'USG'], axis = 0, ascending = False)
usage['rank'] = usage.groupby('Team').rank(method='first', ascending = False)
twitterlineup1 = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\DepthCharts\\lineuptwitter.csv')
twitterlineup2 = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\DepthCharts\\lineup2twitter.csv')
proj_mins = pd.read_excel("C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Projected Minutes\\Export_" + todayds2 + ".xls")
spreads = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Today Spread\\Spread.csv')
opp_ppg = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Opponent Points per game\\Opp_Points_per_Game.csv')
df = pd.read_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Original Files\\masterdf.csv')
todaydvp

,Player,Inj,Likes,Pos,Team,Opp,Def v Pos,VegasPts,Salary,Floor,...,My Proj,Exp.,Used,Con.,Value,Start,Mins,Actual Ownership,Score,Val
0,Russell Westbrook,NaN,NaN,PG,WAS,@ ATL,18th,114.75,11000,51.2,...,65.47160,100,NaN,76,6.0,Y,40.0,NaN,80.25,7.3
1,Giannis Antetokounmpo,NaN,NaN,PF/C,MIL,@ SAS,11th,120.25,10800,43.1,...,57.08170,100,NaN,75,5.3,Y,24.0,NaN,29.75,2.8
2,Domantas Sabonis,NaN,NaN,PF/C,IND,@ CLE,3rd,119.00,10600,45.3,...,57.61600,100,NaN,75,5.4,Y,39.0,NaN,72.50,6.8
3,Stephen Curry,NaN,NaN,PG,GSW,vs UTA,26th,112.00,10500,51.7,...,63.22810,100,NaN,77,6.0,Y,2.0,NaN,7.25,0.7
4,Zion Williamson,NaN,NaN,PF/C,NOP,@ MEM,5th,109.75,9900,0.0,...,0.00000,100,NaN,78,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Zach Collins,NaN,NaN,C,POR,vs HOU,21st,126.75,3000,0.0,...,0.00000,100,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN
200,Derrick Jones Jr.,NaN,NaN,SF,POR,vs HOU,19th,126.75,3000,0.0,...,2.94500,100,NaN,47,1.0,NaN,NaN,NaN,NaN,NaN
201,Nassir Little,NaN,NaN,SF/PF,POR,vs HOU,19th,126.75,3000,0.1,...,9.04412,100,NaN,7,3.0,NaN,NaN,NaN,NaN,NaN
202,D.J. Wilson,NaN,NaN,C,HOU,@ POR,29th,111.75,3000,0.0,...,0.00000,100,NaN,26,0.0,NaN,NaN,NaN,NaN,NaN


In [6]:
# Will take the defense vs position column and transform it into a numerical column

def removeth(th):
    if('st' in th):
        th = int(th.replace('st',''))
    elif('nd' in th):
        th = int(th.replace('nd',''))
    elif('rd' in th):
        th = int(th.replace('rd',''))
    elif('th' in th):
        th = int(th.replace('th',''))
    return th

In [7]:
# Will create one list for players. Will also transform the Defense vs position column and create the
# homevaway column

def dostuff(df):
    players = []
    homevaway = []
    teams = []
    for index, row in df.iterrows():
        if(row['Player'] not in players):
            players.append(row['Player'])
        Opp = row['Opp']
        if('vs' in Opp):
            homevaway.append('home')
        else:
            homevaway.append('away')
        if(row['Team'] not in teams) :
            teams.append(row['Team'])
    df['home_or_away'] = homevaway
    df['DVP'] = df['Def v Pos'].apply(removeth)  
    return players

In [9]:
# Creates a list of all teams

def getteamss(df):
    teams = []
    for index, row in df.iterrows():
        if(row['Team'] not in teams) :
            teams.append(row['Team'])
    return teams

In [10]:
# Will fetch the opp points per game data from the opp_ppg dataset and create a column in our todaydvp dataset

def getoppgcol(df):
    oppg = []
    for index, row in df.iterrows(): 
        home_or_away = row['home_or_away']  
        opp = row['Opp'].split(' ')[1] 
        d = opp_ppg[opp_ppg['Team'] == opp].reset_index()
        if(home_or_away == 'home'):
            d = d.loc[0,'awayppg']
            oppg.append(d)
        else:
            d = d.loc[0,'homeppg']
            oppg.append(d)
    df['oppg'] = oppg

In [12]:
# Uses the master file to create a dataframe for team schedules for all 30 teams in the league

def getteamscheduledf(teams):
    dfteamschedule = pd.DataFrame(columns = ['Team','Dates', 'Game'])
    for t in teams:
        tlist = []
        dates = []
        for index, row in df[df.Team==t].iterrows():
            if(row['Date'] not in dates):
                dates.append(row['Date'])
        games = list(range(1,len(dates) + 1))
        for i in dates:
            tlist.append(t)
        dff = pd.DataFrame()
        dff['Team'] = tlist
        dff['Dates'] = dates
        dff['Game'] = games
        dfteamschedule = pd.concat([dfteamschedule,dff])
    return dfteamschedule

In [13]:
# Takes in a date string and returns the date

def getdate(datestr):
    datet = datetime.strptime(datestr, '%m/%d/%Y')
    return datet

In [14]:
# Adds a game column to the master df

def getgamecoldf(df):
    games = []
    for index, row in df.iterrows():
        p = row['Player']
        datel = datetime.strptime(row['Date'], '%m/%d/%Y')
        teamdat = df[df['Player'] == p].reset_index(drop = True)
        try:
            team = teamdat[teamdat['Date'].apply(getdate) >= datel].reset_index(drop = True).loc[0,'Team']
            gamedat = dfteamschedule[dfteamschedule['Team'] == team].reset_index(drop = True)
            game = gamedat[gamedat['Dates'].apply(getdate) >= datel].reset_index(drop = True).loc[0,'Game']
            games.append(game)
        except KeyError:
            team = teamdat.iloc[-1, 4]
            gamedat = dfteamschedule[dfteamschedule['Team'] == team].reset_index(drop = True)
            game = gamedat.iloc[-1, 2]
            games.append(0)
    df['Game'] = games

In [15]:
# Creates a fuzzy matching function that uses the fuzzywuzzy library and handles exceptions

def getfuzzymatching(playerlist,fuzzedlist,unfuzzedlist):
    list1 = ['Bogdan Bogdanovic','Bojan Bogdanovic','Cole Anthony','Carmelo Anthony','Cam Johnson','James Johnson','Jordan Nwora','Jordan Ford'
         ,'Zion Williamson', 'Lou Williams', 'Marvin Williams', 'Mikal Bridges','Miles Bridges', 'Jalen McDaniels','Jaden McDaniels', 'Derrick Jones Jr.'
         ,'Derrick Rose','Derrick White', 'Patrick Mills', 'Patrick Williams','James Ennis', 'James Harden','Justin James','Justin Patton'
         ,'Anthony Gill', 'Anthony Davis','Jordan Bone', 'Jordan Bell']
    for p in playerlist:
        #for p in players:
        for i in fuzzedlist:
            if(fuzz.partial_ratio(i, p) > 80):
                if( i not in unfuzzedlist and i not in list1 and p not in list1 and p not in unfuzzedlist):
                    unfuzzedlist.append(p)
                elif(i == 'Bogdan Bogdanovic' and p != 'Bojan Bogdanovic'):
                    unfuzzedlist.append(p)
                elif(i == 'Bojan Bogdanovic' and p != 'Bogdan Bogdanovic'):
                    unfuzzedlist.append(p)
                elif(i == 'Cole Anthony' and p != 'Carmelo Anthony'):
                    unfuzzedlist.append(p)
                elif(i == 'Carmelo Anthony' and p != 'Cole Anthony'):
                    unfuzzedlist.append(p)
                elif(i == 'Cam Johnson' and p != 'James Johnson'):
                    unfuzzedlist.append(p)
                elif(i == 'James Johnson' and p != 'Cam Johnson'):
                    unfuzzedlist.append(p)
                elif(i == 'Jordan Ford' and p != 'Jordan Nwora'):
                    unfuzzedlist.append(p)
                elif(i == 'Jordan Nwora' and p != 'Jordan Ford'):
                    unfuzzedlist.append(p)
                elif(i == 'Zion Williamson' and p != 'Lou Williams' and p != 'Marvin Williams'):
                    unfuzzedlist.append(p)
                elif(i == 'Lou Williams' and p != 'Zion Williamson' and p != 'Marvin Williams'):
                    unfuzzedlist.append(p)
                elif(i == 'Marvin Williams' and p != 'Lou Williams' and p != 'Zion Williamson'):
                    unfuzzedlist.append(p)
                elif(i == 'Miles Bridges' and p != 'Mikal Bridges'):
                    unfuzzedlist.append(p)
                elif(i == 'Mikal Bridges' and p != 'Miles Bridges'):
                    unfuzzedlist.append(p)
                elif(i == 'Jalen McDaniels' and p != 'Jaden McDaniels'):
                    unfuzzedlist.append(p)
                elif(i == 'Jaden McDaniels' and p != 'Jalen McDaniels'):
                    unfuzzedlist.append(p)
                elif(i == 'Derrick Jones Jr.' and p != 'Derrick Rose' and p != 'Derrick White'):
                    unfuzzedlist.append(p)
                elif(i == 'Derrick Rose' and p != 'Derrick Jones Jr.' and p != 'Derrick White'):
                    unfuzzedlist.append(p)
                elif(i == 'Derrick White' and p != 'Derrick Rose' and p != 'Derrick Jones Jr.'):
                    unfuzzedlist.append(p)
                elif(i == 'Patrick Mills' and p != 'Patrick Williams'):
                    unfuzzedlist.append(p)
                elif(i == 'Patrick Williams' and p != 'Patrick Mills'):
                    unfuzzedlist.append(p)
                elif(i == 'James Ennis' and p != 'James Harden'):
                    unfuzzedlist.append(p)
                elif(i == 'James Harden' and p != 'James Ennis'):
                    unfuzzedlist.append(p)
                elif(i == 'Justin Patton' and p != 'Justin James'):
                    unfuzzedlist.append(p)
                elif(i == 'Justin James' and p != 'Justin Patton'):
                    unfuzzedlist.append(p)
                elif(i == 'Anthony Gill' and p != 'Anthony Davis'):
                    unfuzzedlist.append(p)
                elif(i == 'Anthony Davis' and p != 'Anthony Gill'):
                    unfuzzedlist.append(p)
                elif(i == 'Jordan Bone' and p != 'Jordan Bell'):
                    unfuzzedlist.append(p)
                elif(i == 'Jordan Bell' and p != 'Jordan Bone'):
                    unfuzzedlist.append(p)
                    
    if('Cam Payne' in fuzzedlist and 'Cameron Payne' not in unfuzzedlist and 'Cameron Payne' in playerlist):
        unfuzzedlist.append('Cameron Payne')
    elif('Cam Johnson' in fuzzedlist and 'Cameron Johnson' not in unfuzzedlist and 'Cameron Johnson' in playerlist):
        unfuzzedlist.append('Cameron Johnson')
    elif('Ish Smith' in fuzzedlist and 'Ishmael Smith' not in unfuzzedlist and 'Ishmael Smith' in playerlist):
        unfuzzedlist.append('Ishmael Smith')
    elif('Raul Neto' in fuzzedlist and 'Raulzinho Neto' not in unfuzzedlist and 'Raulzinho Neto' in playerlist):
        unfuzzedlist.append('Raulzinho Neto')
    elif('Willy Hernangomez' in fuzzedlist  and 'Guillermo Hernangomez' not in unfuzzedlist and 'Guillermo Hernangomez' in playerlist):
        unfuzzedlist.append('Guillermo Hernangomez')
    elif('Patty Mills' in fuzzedlist  and 'Patrick Mills' not in unfuzzedlist and 'Patrick Mills' in playerlist):
        unfuzzedlist.append('Patrick Mills')
    elif('Lu Dort' in fuzzedlist  and 'Luguentz Dort' not in unfuzzedlist and 'Luguentz Dort' in playerlist):
        unfuzzedlist.append('Luguentz Dort')
    elif('Mo Bamba' in fuzzedlist  and 'Mohamed Bamba' not in unfuzzedlist and 'Mohamed Bamba' in playerlist):
        unfuzzedlist.append('Mohamed Bamba')
    elif('Nic Claxton' in fuzzedlist  and 'Nicolas Claxton' not in unfuzzedlist and 'Nicolas Claxton' in playerlist):
        unfuzzedlist.append('Nicolas Claxton')
    elif("De’Anthony Melton" in fuzzedlist  and "De'Anthony Melton" not in unfuzzedlist and "De'Anthony Melton" in playerlist):
        unfuzzedlist.append("De'Anthony Melton")
    elif("Wes Iwundu" in fuzzedlist  and "Wesley Iwundu" not in unfuzzedlist and "Wesley Iwundu" in playerlist):
        unfuzzedlist.append("Wesley Iwundu")
    elif("Moe Wagner" in fuzzedlist  and "Moritz Wagner" not in unfuzzedlist and "Moritz Wagner" in playerlist):
        unfuzzedlist.append("Moritz Wagner")
    elif("Shaq Harrison" in fuzzedlist  and "Shaquille Harrison" not in unfuzzedlist and "Shaquille Harrison" in playerlist):
        unfuzzedlist.append("Shaquille Harrison")
    elif('Cam Reynolds' in fuzzedlist and 'Cameron Reynolds' not in unfuzzedlist and 'Cameron Reynolds' in playerlist):
        unfuzzedlist.append('Cameron Reynolds')
    elif('Cristiano Felicio' in fuzzedlist and 'Cristiano Da Silva Felicio' not in unfuzzedlist and 'Cristiano Da Silva Felicio' in playerlist):
        unfuzzedlist.append('Cristiano Da Silva Felicio')

In [16]:
# Will be used to remove leading and trailing whitespaces

def strip(name):
    name = name.strip()
    return name

In [17]:
# Used to get the matched names of the injured players for our todaydvp df, since twitter and fantasy cruncher use different 
# naming conventions

def getmatchedinjuries(injuries, injuriestwitter, players):
    injurieslist = list(injuries.iloc[:,0].apply(strip))
    twitterinjurieslist = list(injuriestwitter.iloc[:,0].apply(strip))
    for p in twitterinjurieslist:
        if(p.strip() not in injurieslist):
            injurieslist.append(p.strip())
    injurycheck = []
    for i in range(0,5):  
        getfuzzymatching(players,injurieslist,injurycheck)
    injurycheck = list(OrderedDict.fromkeys(injurycheck)) 
    return injurycheck

In [18]:
# Used matchedinjuries list to create a column in our todaydvp df to indicate if a player is injured or not

def getisinjuredcol(todaydvp):
    isinjured = []
    for index, row in todaydvp.iterrows():
        player  = row['Player']
        if(player in injurycheck):
            isinjured.append(True)
        else:
            isinjured.append(False)
    todaydvp['isinjured'] = isinjured

In [19]:
# Creates an isstarter column in our todaydvp df and returns the starter list

def getstarters(players, todaydvp):
    startersunmatched = list(common_lineups[common_lineups['Depth'] <6].loc[:,'Name'].apply(strip))
    starters = []
    getfuzzymatching(players,startersunmatched,starters)
    isstarter = []
    for index, row in todaydvp.iterrows():
        if (row['Player'] in starters):
            isstarter.append(True)
        else:
            isstarter.append(False)
    todaydvp['isstarter'] =  isstarter
    return starters

In [20]:
# Creates an isstud column in our todaydvp df and returns the stud list (identified by their usage percentage on the team - has to be 1st or 2nd ranked
# for usage). Additional exceptional players are also added (identified by subject matter expert fantasy player)

def getstuds(usage, todaydvp):
    studs = list(usage[usage['rank'] <= 2].loc[:,'Name'])
    studs.append('James Harden')
    studs.append('Clint Capela')
    studs.append('Kemba Walker')
    studs.append('Domantas Sabonis')
    studs.append('Jusuf Nurkic')
    studs.append('Richaun Holmes')
    studs.append('Ben Simmons')
    studs.append('Jrue Holiday')
    isstud = []
    for index, row in todaydvp.iterrows():
        if (row['Player'] in studs):
            isstud.append(True)
        else:
            isstud.append(False)
    todaydvp['isstud'] =  isstud
    return studs

In [21]:
# Cleans up the pos column in our todaydvp column and creates a bigsmall column to differentiate between bigger positions (less 
# likely to score more points) and smaller positions (more likely to score more)

def getpos(todaydvp, common_lineups):
    smalls = ['PG/SF','PG','SG/SF','SG','PG/SG','SF']
    bigs = ['PF/C','PF','C']
    pos = []
    for index, row in todaydvp.iterrows():
        p = row['Player']
        if(row['Pos'] == 'SF/PF'):
            try:
                d = common_lineups[common_lineups['Name'] == p]
                pos.append(d.reset_index(drop = True).loc[0,'Pos'])
            except:
                pos.append('SF')
        else:
            pos.append(row['Pos'])
    todaydvp['Position'] = pos
    bigsmall = []
    for index, row in todaydvp.iterrows():
        if(row['Position'] in smalls):
            bigsmall.append('Small')
        else:
            bigsmall.append('Big')
    todaydvp['BigSmall'] = bigsmall

In [22]:
# Gets an initial list of unmatched backups (meaning the naming conventions are probably different than in our todaydvp df)

def getbackups(todaydvp, depth_charts, common_lineups2): 
    todaydvpstarters = todaydvp.loc[(todaydvp['isstarter'] == True) & (todaydvp['isinjured']  == True)]
    todaydvpinj = todaydvp[todaydvp['isinjured'] == True]
    injuredstarternames = list(todaydvpstarters['Player'])
    backup = []
    teams = []
    for i in injuredstarternames:
        d = depth_charts[depth_charts['Starters'] == i]
        back = d.reset_index(drop = True).loc[0,'Seconds']
        if(back in starters):
            bigsmall = todaydvp[todaydvp['Player'] == i].reset_index(drop = True).loc[0,'BigSmall']
            team = todaydvp[todaydvp['Player']==i].reset_index(drop = True).loc[0,'Team']
            ind = 0
            d = common_lineups2.loc[(common_lineups2.Team == team) & (common_lineups2.BigSmall == bigsmall)]
            back = d.reset_index(drop = True).loc[0,'Name']
            if(back in backup or back in injurycheck):
                ind +=1
                back = d.reset_index(drop = True).loc[ind,'Name']
                if(back in backup or back in injurycheck):
                    ind +=1
                    back = d.reset_index(drop = True).loc[ind,'Name']
                    if(back in backup or back in injurycheck):
                        ind +=1
                        back = d.reset_index(drop = True).loc[ind,'Name']
                        if(back in backup or back in injurycheck):
                            ind +=1
                            back = d.reset_index(drop = True).loc[ind,'Name']
                            if(back in backup or back in injurycheck):
                                ind +=1
                                back = d.reset_index(drop = True).loc[ind,'Name']
                            else:
                                backup.append(back)
                        else:
                            backup.append(back)
                    else:
                        backup.append(back)
                else:
                    backup.append(back)
            else:
                backup.append(back)
        else:
            backup.append(back)
    return backup

In [23]:
# Creates a lookup df for players and their last names used to look up backups and guys that are starting in place of an 
# injured starter

def getplayernamelookup(players, todaydvp):
    playerlastnames = []
    playernames = []
    teams = []
    for i in players:
        playernames.append(i)
        playerlastnames.append(i.split(' ')[1])
        teams.append(todaydvp[todaydvp.Player == i].reset_index(drop = True).loc[0,'Team'])
        
    dfplays = pd.DataFrame()
    dfplays['Player'] = playernames
    dfplays['Last'] = playerlastnames
    dfplays['Team'] = teams
    return dfplays

In [24]:
# Gets the correct naming conventions for backups and guys that are starting in place of an injured starter. Also creates a list
# of newly starting PGs (to provide a boost to their projection later on for a competetive advantage in building lineups)

def getmatchedlinpgs(twitterlineup1, dfplays, twitterlineup2, backup):
    matchedlineups = []
    matchedpgs = []
    for index, row in twitterlineup1.iterrows():
        team = row['Team']
        last = row['Player']
        pg = row['Is PG']
        try:
            d = dfplays.loc[(dfplays['Last'] == last) & (dfplays['Team'] == team)].reset_index(drop = True).loc[0,'Player']
            if(pg == 1):
                d2 = dfplays.loc[(dfplays['Last'] == last) & (dfplays['Team'] == team)].reset_index(drop = True).loc[0,'Player']
                matchedpgs.append(d2)
            matchedlineups.append(d)
        except KeyError:
            pass
    lineups2 = list(twitterlineup2.loc[:,"Player"].unique())
    for index, row in twitterlineup1.iterrows():
        team = row['Team']
        last = row['Player']
        if(len(last.split(' ')) > 1):
            lineups2.append(last)
    matchedlineups2 = []
    for i in range(0,5):  
        getfuzzymatching(players,lineups2,matchedlineups2)
    matchedlineups2 = list(OrderedDict.fromkeys(matchedlineups2)) 
    for i in matchedlineups2:
        if(i not in matchedlineups):
            matchedlineups.append(i)
    for i in injurycheck:
        if i in matchedlineups:
            injurycheck.remove(i)
    for i in matchedlineups:
        if(i not in starters and i not in backup):
            backup.append(i)
    backups = []
    for i in range(0,5):  
        getfuzzymatching(players,backup,backups)
    backups = list(OrderedDict.fromkeys(backups)) 
    return matchedlineups, matchedpgs, backups

In [25]:
# Gets updated (latest) projected mins and USG percentage to use when building predictions. 

def getprojminsandusage(proj_mins, players, todaydvp):
    firsts = []
    lasts = []
    for index, row in proj_mins.iterrows():
        name = row['Name']
        first = name.split(' ')[0]
        last = name.split(' ')[1]
        firsts.append(first)
        lasts.append(last)
    proj_mins['Last'] = lasts
    proj_mins['First'] = firsts
    proj_mins_list = []
    USGS = []
    for p in players:
        last = p.split(' ')[1]
        team = todaydvp[todaydvp.Player == p].reset_index(drop = True).loc[0,'Team']
        if(team == 'PHX'):
            team = 'PHO'
        d = proj_mins[proj_mins.Team == team]
        dlast = d[d.Last == last]
        if(len(dlast) > 0):
            if(len(dlast) > 1):
                dfirst = dlast[dlast.First == p.split(' ')[0]]
                proj_mins_list.append(dfirst.reset_index(drop = True).loc[0, 'm/g'])
                USGS.append(dfirst.reset_index(drop = True).loc[0, 'USG'])
            else:
                proj_mins_list.append(dlast.reset_index(drop = True).loc[0, 'm/g'])
                USGS.append(dlast.reset_index(drop = True).loc[0, 'USG'])
        else:
            d = todaydvp[todaydvp['Player'] == p].reset_index(drop = True).loc[0,'Proj Mins']
            u = todaydvp[todaydvp['Player'] == p].reset_index(drop = True).loc[0,'USG']
            proj_mins_list.append(d)
            USGS.append(u)
    todaydvp['Proj_mins'] = proj_mins_list
    todaydvp['USGS'] = USGS

In [26]:
# Creates a dataframe for teams and the number of injured starters and injured studs they have

def getinjstartandstudsdf(todaydvp):
    teamlist = []
    for index, row in todaydvp.iterrows():
        teamlist.append(row['Team'])     
    teamlist = list(OrderedDict.fromkeys(teamlist)) 
    todaydvpstarters = todaydvp.loc[(todaydvp['isstarter'] == True) & (todaydvp['Proj_mins']  == 0)]
    todaydvpstuds = todaydvp.loc[(todaydvp['isstud'] == True) & (todaydvp['Proj_mins']  == 0)]
    injuredstuds = []
    injuredstarters = []
    for t in teamlist:
        d = todaydvpstarters[todaydvpstarters['Team'] == t]
        d2 = todaydvpstuds[todaydvpstuds['Team'] == t]
        injuredstarters.append(len(d))
        injuredstuds.append(len(d2))
    dfinjstart = pd.DataFrame()
    dfinjstart['Team'] = teamlist
    dfinjstart['InjuredStarters'] = injuredstarters
    dfinjstart['InjuredStuds'] = injuredstuds
    return dfinjstart

In [27]:
# Creates columns for number of injured starters and injured studs in our todaydvp df

def getinjstartsandstuds(todaydvp, dfinjstart):
    studinj = []
    startinj = []
    for index, row in todaydvp.iterrows():
        t = row['Team']
        data = dfinjstart[dfinjstart['Team'] == t].reset_index(drop = True).loc[0,'InjuredStarters']
        data2 = dfinjstart[dfinjstart['Team'] == t].reset_index(drop = True).loc[0,'InjuredStuds']
        startinj.append(data)
        studinj.append(data2)
    todaydvp['InjuredStarters'] = startinj
    todaydvp['InjuredStuds'] = studinj

In [28]:
# Creates a column to indicate the last game that the player's team played (used later to calculate last 5 game fantasy pt avg)

def getlastgamescol(todaydvp):
    lastgames = []
    for index, row in todaydvp.iterrows():
        team = row['Team']
        lastgame = dfteamschedule[dfteamschedule['Team'] == team].loc[:,'Game'].max()
        lastgames.append(lastgame)
    todaydvp['LastGame'] = lastgames

In [29]:
''' 
Creates columns for the todaydvp df capturing the players avg fantasy points and mins in the last 5 games (if player hasn't 
played in the last 5 games it grabs their last 5 recorded stats).It also creates a column for avg fantasy points against 
the opp for that week (if none it pulls last 3 game avg fantasy points).It creates a column for overall mins and overall
points averages for the player (for the season), creates a points per min column, and uses the predicted mins and the points 
per min calculation to create a ppm projection for the game. Backtoback identifies games where the player played the previous
day.Finally it creates a column for the players' team average pace and the players' opp average pace using the pace dataset. 
'''
def getmorecols(todaydvp):
    avgfps = []
    avgmins = []
    avgfpsopp = []
    avgteampace = []
    avgopppace = []
    backtoback = []
    ovravg = []
    ovrmin = []
    for index, row in todaydvp.iterrows():
        p = row['Player']
        gamelower = row['LastGame'] - 4 #-4
        gameupper = row['LastGame'] 
        opp = row['Opp'].split(' ')[1]
        team = row['Team']
        pred_mins = row['Proj_mins']
        data = df[df['Player'] == p]
        data2 = data.loc[(data['Game'] >= gamelower) & (data['Game'] <= gameupper)]
        avgfp = data2.loc[:,'Score'].median()
        avgmin = data2.loc[:,'Mins'].median()
        if(math.isnan(avgfp) == True):
            avgfp = data['Score'].dropna()[-5:].median()
            if(math.isnan(avgfp) == True):
                avgfp = 0
        if(math.isnan(avgmin) == True):
            avgmin = data['Mins'].dropna()[-5:].median()
            if(math.isnan(avgmin) == True):
                avgmin = 0
        avgfps.append(avgfp)
        avgmins.append(avgmin)
        oppdata = data.query('Opp.str.contains(@opp) ', engine = 'python')
        oppdata = oppdata.loc[:,'Score']
        avgagainstoppp = oppdata.mean()
        if(math.isnan(avgagainstoppp) == True):
            gamelower = row['LastGame'] - 3
            data2 = data.loc[(data['Game'] >= gamelower) & (data['Game'] <= gameupper)]
            avgagainstoppp = data2.loc[:,'Score'].median()
            if(math.isnan(avgagainstoppp) == True):
                avgagainstoppp = data['Score'].dropna()[-3:].median()
                if(math.isnan(avgagainstoppp) == True):
                   avgagainstoppp = 0
        avgfpsopp.append(avgagainstoppp)
        teamd = Pace[Pace.ABR == team].reset_index(drop = True)
        teampace = teamd.loc[0,'Pace']
        oppd = Pace[Pace.ABR == opp].reset_index(drop = True)
        opppace = oppd.loc[0,'Pace']
        avgteampace.append(teampace)
        avgopppace.append(opppace)
        d = data[data['Date'].apply(getdate) == str(yesterdate)]
        if(len(d) > 0 or (p in injurycheck and pred_mins == 0) ):
            backtoback.append(True)
        else:
            backtoback.append(False)
        avg = data.loc[:,'Score'].mean()
        avg2 = data.loc[:,'Mins'].mean()
        if(math.isnan(avg)):
            ovravg.append(0)
        else:
            ovravg.append(avg)
        if(math.isnan(avg2)):
            ovrmin.append(0)
        else:
            ovrmin.append(avg2)
    todaydvp['AveragePointsLast5Games'] = avgfps
    todaydvp['AverageMinsLast5Games'] = avgmins
    todaydvp['Averagefantasypointagainstopp'] = avgfpsopp
    todaydvp['Averageteampace'] = avgteampace
    todaydvp['Averageopppace']  =  avgopppace
    todaydvp['BacktoBack'] = backtoback
    todaydvp['ovravg'] = ovravg
    todaydvp['ovrmins'] = ovrmin
    todaydvp['PPM'] = todaydvp['ovravg'] / todaydvp['ovrmins']
    todaydvp['PPM proj'] = todaydvp['PPM'] * todaydvp['Proj_mins']

In [30]:
# Uses the loaded scaler to scaler the features

def scalefeatures(scaler, X):
    X_scaled = scaler.transform(X)
    return X_scaled

In [31]:
# Adds a prediction and opp column to the final df

def addpredsandoppfinal(final, y_pred, todaydvp):
    final['Prediction'] = y_pred
    final['Opp'] = todaydvp['Opp']

In [32]:
# Adjusts the original prediction. If a player has 0 predicted mins, their prediction becomes 0. Players that are not direct 
# backups or starters are reduced in order to prevent them from entering lineups. PGs starting for an inj starter get a 25%
# boost to provide a competetive advantage when lineup building. 

def adjustpredictions(final, backups, starters, matchedpgs):
    newprojection = []
    for index, row in final.iterrows():
        if(row['Player'] not in backups and row['Player'] not in starters):
            newprojection.append(round(row['Prediction'],2) * .75)
        elif(row['Proj_mins'] == 0):
            newprojection.append(0)
        elif(row['Player'] in backups and row['Player'] in matchedpgs):
            newprojection.append(round(row['Prediction'],2) * 1.25)
        else:
            newprojection.append(round(row['Prediction'],2))
    return newprojection

In [33]:
# Drops unadjusted predictions, moves adjusted predictions to the front, rounds all numerical values, and sorts highest to 
# lowest predictions

def finalcleanup(final, newprojection):
    final['Predictions'] = newprojection
    final = final.drop('Prediction', axis = 1)
    final = final.sort_values("Predictions", ascending = False)
    final = final.round(2)
    final.insert(loc=1, column = 'Preds', value = final['Predictions'])
    final = final.drop('Predictions', axis = 1)
    return final

# Load lists and columns

In [34]:
players, teams = dostuff(todaydvp), getteamss(df)                    # Creates player and teams list
getoppgcol(todaydvp)                                                 # Adds oppg column to todaydvp df               
dfteamschedule = getteamscheduledf(teams)                            # Creates df for team schedules
getgamecoldf(df)                                                     # Adds game column to master df
injurycheck =  getmatchedinjuries(injuries, injuriestwitter, players)  # Gets Injuries for today
getisinjuredcol(todaydvp)                                            # Adds isinjured col to todaydvp df
starters = getstarters(players, todaydvp)                            # Creates starter list and isstarter column
studs = getstuds(usage, todaydvp)                                    # Creates stud list and isstud column
getpos(todaydvp, common_lineups)                                     # Cleans up position column in todaydvp df
backup = getbackups(todaydvp, depth_charts, common_lineups2)         # Get unmatched backup list
dfplays = getplayernamelookup(players, todaydvp)                     # Creates lookup dataframe for incorrectly formatted names
matchedlineups, matchedpgs, backups = getmatchedlinpgs(twitterlineup1, dfplays, twitterlineup2, backup) #Returns list of guys starting for injured starter, newly starting pgs, and backups with matching naming convention
getprojminsandusage(proj_mins, players, todaydvp)                    # Gets latest projected mins and usage
dfinjstart = getinjstartandstudsdf(todaydvp)                     # Creates a df for teams indicating # of inj Starters and studs
getinjstartsandstuds(todaydvp, dfinjstart)                           # Creates columns in todaydvp df for inj starters and studs
getlastgamescol(todaydvp)                                           # Creates column for last game teams have played in todaydvp
getmorecols(todaydvp)                                               # Gets remaining columns specified in the function

# Extracting features to be used to predict scores

In [35]:
new_df =  todaydvp[['Player','Team','VegasPts','Salary','Floor','Ceiling','STDV', 'USGS','Proj_mins','Con.','home_or_away','DVP','oppg','isstarter','isstud'
               ,'isinjured','BacktoBack','InjuredStarters','InjuredStuds','AveragePointsLast5Games','AverageMinsLast5Games','Averagefantasypointagainstopp'
               ,'ovravg','ovrmins','PPM proj','Averageteampace','Averageopppace']] 
new_df = new_df.drop('Team', axis = 1)
final = pd.get_dummies(new_df, columns = ['home_or_away','isstarter','isstud','isinjured','BacktoBack'], drop_first = True)
Xdf = final.drop('Player', axis = 1).fillna(0)
X = Xdf.values
Xdf.shape 

(204, 25)

In [36]:
final.iloc[:10, :13] # Look at first 13 columns of final

,Player,VegasPts,Salary,Floor,Ceiling,STDV,USGS,Proj_mins,Con.,DVP,oppg,InjuredStarters,InjuredStuds
0,Russell Westbrook,114.75,11000,51.2,86.8,14.24,33.933460,40.0,76,18,109.8,2,1
1,Giannis Antetokounmpo,120.25,10800,43.1,78.1,14.02,33.090797,33.0,75,11,112.5,0,0
2,Domantas Sabonis,119.00,10600,45.3,76.0,12.27,24.894655,38.0,75,3,114.9,1,1
3,Stephen Curry,112.00,10500,51.7,80.6,11.57,35.417926,37.0,77,26,110.9,0,0
4,Zion Williamson,109.75,9900,0.0,14.4,9.61,29.900000,0.0,78,5,110.8,3,2
5,Damian Lillard,126.75,9700,45.6,75.4,11.93,29.235275,36.0,75,25,118.0,0,0
6,Donovan Mitchell,114.00,9600,0.0,15.2,10.10,0.000000,0.0,76,15,111.1,2,1
7,Bradley Beal,114.75,9400,0.0,16.5,10.99,0.000000,0.0,76,19,109.8,2,1
8,Trae Young,122.75,9300,41.2,69.3,11.23,29.275377,35.0,76,13,117.8,0,0
9,Caris LeVert,119.00,9000,33.4,66.7,13.32,27.579762,37.0,64,2,114.9,1,1


In [37]:
final.iloc[:10, 13:] # Look at last 13 columns of final

,AveragePointsLast5Games,AverageMinsLast5Games,Averagefantasypointagainstopp,ovravg,ovrmins,PPM proj,Averageteampace,Averageopppace,home_or_away_home,isstarter_True,isstud_True,isinjured_True,BacktoBack_True
0,81.000,40.0,60.875,58.654167,36.433333,64.396157,106.4,99.8,0,1,1,0,0
1,47.750,29.5,48.625,55.236364,32.690909,55.758621,104.3,100.9,0,1,1,0,0
2,72.500,41.0,53.500,49.401786,36.017857,52.120476,103.9,99.5,0,1,1,0,0
3,59.250,35.0,51.500,51.286885,34.016393,55.785301,104.4,101.0,1,1,1,0,0
4,50.250,37.0,48.125,43.720833,33.100000,0.000000,102.2,102.7,0,1,1,1,1
5,55.250,34.0,53.000,48.472222,35.714286,48.860000,100.9,103.6,1,1,1,0,0
6,57.000,35.0,36.625,43.397059,33.705882,0.000000,101.0,104.4,0,1,1,1,1
7,49.625,39.0,38.500,46.377193,35.719298,0.000000,106.4,99.8,0,1,1,1,1
8,51.250,35.0,54.125,46.482759,34.068966,47.753036,99.8,106.4,1,1,1,0,0
9,52.250,38.0,44.750,37.470238,31.476190,44.045953,103.9,99.5,0,1,1,0,0


# Load the Saved Model and Scaler

In [38]:
from tensorflow.keras.models import save_model, load_model
filepath = 'C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Python Programs\\test model3\\'
model = load_model(filepath)

with open('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Python Programs\\test model3\\scaler','rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# Scale the features 

In [39]:
X_scaled = scalefeatures(scaler, X)

# Generate Predictions with Loaded Model

In [40]:
y_pred = model.predict(X_scaled)

In [41]:
# Add the predictions and opp to our final df, adjust predictions and clean the final df

addpredsandoppfinal(final, y_pred, todaydvp)
newprojection = adjustpredictions(final, backups, starters, matchedpgs)
final = finalcleanup(final, newprojection)

In [42]:
final.iloc[:10, :13] # Look at first 13 columns of final

,Player,Preds,VegasPts,Salary,Floor,Ceiling,STDV,USGS,Proj_mins,Con.,DVP,oppg,InjuredStarters
0,Russell Westbrook,63.12,114.75,11000,51.2,86.8,14.24,33.93,40.0,76,18,109.8,2
3,Stephen Curry,55.63,112.00,10500,51.7,80.6,11.57,35.42,37.0,77,26,110.9,0
1,Giannis Antetokounmpo,54.32,120.25,10800,43.1,78.1,14.02,33.09,33.0,75,11,112.5,0
2,Domantas Sabonis,52.55,119.00,10600,45.3,76.0,12.27,24.89,38.0,75,3,114.9,1
5,Damian Lillard,52.33,126.75,9700,45.6,75.4,11.93,29.24,36.0,75,25,118.0,0
8,Trae Young,50.84,122.75,9300,41.2,69.3,11.23,29.28,35.0,76,13,117.8,0
18,Jonas Valanciunas,44.17,118.75,8200,29.8,58.0,11.30,22.42,33.0,71,9,115.5,1
24,DeMar DeRozan,43.61,113.25,7900,33.6,62.0,11.35,24.69,35.0,71,30,114.8,1
9,Caris LeVert,43.40,119.00,9000,33.4,66.7,13.32,27.58,37.0,64,2,114.9,1
25,C.J. McCollum,43.36,126.75,7800,29.4,56.3,10.76,26.11,36.0,72,22,118.0,0


We have our predictions (preds column). We have successfully loaded and used the model to predict fantasy scores for new data!

In [44]:
# Save the dataframe to a csv!
final.to_csv('C:\\Users\\v-mpurvis\\OneDrive\\Personal Files\\Fantasy Basketball\\Predictions\\NBA_Fantasy_Scores_Predictions_test.csv', index = False)